In [1]:
import pandas as pd
from os.path import join, isfile
from os import listdir
import numpy as np

In [3]:
datafiles = [join('.\\data', f) for f in listdir('./data') if join('./data', f).endswith(".csv")]

In [5]:
datafiles

['.\\data\\Guess_1167_part0_2025-07-02_18h08.55.968.csv',
 '.\\data\\Guess_1167_part1_part2_2025-07-02_18h32.03.916.csv',
 '.\\data\\Guess_1167_sm_2025-07-02_19h13.56.366.csv',
 '.\\data\\Guess_2172_part0_2025-06-23_15h31.44.862.csv',
 '.\\data\\Guess_2172_part1_part2_2025-06-23_15h46.32.531.csv',
 '.\\data\\Guess_2172_sm_2025-06-23_16h27.15.471.csv',
 '.\\data\\Guess_2405_part0_2025-06-18_18h53.06.633.csv',
 '.\\data\\Guess_2405_part1_part2_2025-06-18_19h05.06.793.csv',
 '.\\data\\Guess_2405_sm_2025-06-18_19h52.42.236.csv',
 '.\\data\\Guess_3084_part0_2025-05-30_12h54.23.016.csv',
 '.\\data\\Guess_3084_part0_2025-05-30_13h02.37.105.csv',
 '.\\data\\Guess_3084_part1_part2_2025-05-30_13h16.54.106.csv',
 '.\\data\\Guess_3084_sm_2025-05-30_14h00.35.729.csv',
 '.\\data\\Guess_4110_part0_2025-06-06_12h56.37.524.csv',
 '.\\data\\Guess_4110_part1_part2_2025-06-06_13h16.02.627.csv',
 '.\\data\\Guess_4110_sm_2025-06-06_14h00.43.501.csv',
 '.\\data\\Guess_5613_part0_2025-06-04_17h50.35.863.csv',

In [7]:
counter = 0
learning_frames = []
recall_frames = []

for f in datafiles:
    print(f)
    if "end_part1_text.started" in pd.read_csv(f):
        df = pd.read_csv(f, converters={'PID': str})
             
        df["subj_recalled"] = 0 # column in learning for when participants responded with recall
        df["corr_recalled"] = 0 # column for when participants correctly chose the last letter

        for row in df.loc[df["test_trials.thisIndex"] >= 0].iterrows():
            subj_recall = row[1]["recall_reached.keys"]
            corr_recall = row[1]["correct_choice"]
            cue =  row[1]["Cue"]
            if subj_recall == 3:
                df.loc[df["Cue"] == cue, "subj_recalled"] = 1
            else:
                df.loc[df["Cue"] == cue, "subj_recalled"] = 0
            
            if corr_recall == 1:
                df.loc[df["Cue"] == cue, "corr_recalled"] = 1
            else:
                df.loc[df["Cue"] == cue, "corr_recalled"] = 0

        # LEARNING
        learning = df.loc[df["learning_trials.thisN"] >= 0]
        learning_frames.append(learning)

        
        # RECALL
        recall = df.loc[df["test_trials.thisN"] >= 0]
        recall_frames.append(recall)
        counter += 1
print('Number of finished experiments', counter)

.\data\Guess_1167_part0_2025-07-02_18h08.55.968.csv
.\data\Guess_1167_part1_part2_2025-07-02_18h32.03.916.csv
.\data\Guess_1167_sm_2025-07-02_19h13.56.366.csv
.\data\Guess_2172_part0_2025-06-23_15h31.44.862.csv
.\data\Guess_2172_part1_part2_2025-06-23_15h46.32.531.csv
.\data\Guess_2172_sm_2025-06-23_16h27.15.471.csv
.\data\Guess_2405_part0_2025-06-18_18h53.06.633.csv
.\data\Guess_2405_part1_part2_2025-06-18_19h05.06.793.csv
.\data\Guess_2405_sm_2025-06-18_19h52.42.236.csv
.\data\Guess_3084_part0_2025-05-30_12h54.23.016.csv
.\data\Guess_3084_part0_2025-05-30_13h02.37.105.csv
.\data\Guess_3084_part1_part2_2025-05-30_13h16.54.106.csv
.\data\Guess_3084_sm_2025-05-30_14h00.35.729.csv
.\data\Guess_4110_part0_2025-06-06_12h56.37.524.csv
.\data\Guess_4110_part1_part2_2025-06-06_13h16.02.627.csv
.\data\Guess_4110_sm_2025-06-06_14h00.43.501.csv
.\data\Guess_5613_part0_2025-06-04_17h50.35.863.csv
.\data\Guess_5613_part1_part2_2025-06-04_18h20.36.357.csv
.\data\Guess_5613_sm_2025-06-04_19h01.43.63

In [9]:
learning_frames[0]["corr_recalled"]

11    1
12    1
13    1
14    1
15    1
     ..
86    0
87    0
88    1
89    1
90    0
Name: corr_recalled, Length: 80, dtype: int64

In [11]:
delim = ' '
form = '%.5f'
# guess_dur = 3 # none in final version
guess_react_dur = 3
encoding_dur = 3

for frame in learning_frames:
    PID = frame["PID"].values[0]
    triggers = frame["wait_for_trigger.stopped"][frame["wait_for_trigger.stopped"].notna()].values
    print(PID)
    print(triggers)
    guess_reached_times1 = frame.loc[frame["trial_type"]=="Guess","guess_response.started"] - triggers[0]
    guess_reached_times2 = frame.loc[frame["trial_type"]=="Guess","guess_response.started"] - triggers[1]
    guess_reached_times1 = guess_reached_times1.values[guess_reached_times2.values < 0]
    guess_reached_times2 = guess_reached_times2.values[guess_reached_times2.values >= 0]
    guess_reached_times1 = np.array([guess_reached_times1, np.ones(len(guess_reached_times1)) * guess_react_dur, np.ones(len(guess_reached_times1))]).T
    guess_reached_times2 = np.array([guess_reached_times2, np.ones(len(guess_reached_times2)) * guess_react_dur, np.ones(len(guess_reached_times2))]).T
    np.savetxt('onset_files/' + PID + '_guess_response_onset_run1.txt', guess_reached_times1, fmt=form, delimiter=delim)
    np.savetxt('onset_files/' + PID + '_guess_response_onset_run2.txt', guess_reached_times2, fmt=form, delimiter=delim)

    # Guess VS Read
    encode_guess_times1 = frame.loc[frame["trial_type"]=="Guess","encode.started"] - triggers[0]
    encode_guess_times2 = frame.loc[frame["trial_type"]=="Guess","encode.started"] - triggers[1]
    encode_guess_times1 = encode_guess_times1.values[encode_guess_times2.values < 0]
    encode_guess_times2 = encode_guess_times2.values[encode_guess_times2.values >= 0]
    encode_guess_times1 = np.array([encode_guess_times1, np.ones(len(encode_guess_times1)) * encoding_dur, np.ones(len(encode_guess_times1))]).T
    encode_guess_times2 = np.array([encode_guess_times2, np.ones(len(encode_guess_times2)) * encoding_dur, np.ones(len(encode_guess_times2))]).T
    np.savetxt('onset_files/' + PID + '_encode_guess_onset_run1.txt', encode_guess_times1, fmt=form, delimiter=delim)
    np.savetxt('onset_files/' + PID + '_encode_guess_onset_run2.txt', encode_guess_times2, fmt=form, delimiter=delim)

    encode_read_times1 = frame.loc[frame["trial_type"]=="Read","encode.started"] - triggers[0]
    encode_read_times2 = frame.loc[frame["trial_type"]=="Read","encode.started"] - triggers[1]
    encode_read_times1 = encode_read_times1.values[encode_read_times2.values < 0]
    encode_read_times2 = encode_read_times2.values[encode_read_times2.values >= 0]
    encode_read_times1 = np.array([encode_read_times1, np.ones(len(encode_read_times1)) * encoding_dur, np.ones(len(encode_read_times1))]).T
    encode_read_times2 = np.array([encode_read_times2, np.ones(len(encode_read_times2)) * encoding_dur, np.ones(len(encode_read_times2))]).T
    np.savetxt('onset_files/' + PID + '_encode_read_onset_run1.txt', encode_read_times1, fmt=form, delimiter=delim)
    np.savetxt('onset_files/' + PID + '_encode_read_onset_run2.txt', encode_read_times2, fmt=form, delimiter=delim)

    # Subsequent memory
    # Guess
    encode_guess_correct_times1 = frame.loc[((frame["trial_type"]=="Guess") & (frame["corr_recalled"]==1)),"encode.started"] - triggers[0]
    encode_guess_correct_times2 = frame.loc[((frame["trial_type"]=="Guess") & (frame["corr_recalled"]==1)),"encode.started"] - triggers[1]
    encode_guess_correct_times1 = encode_guess_correct_times1.values[encode_guess_correct_times2.values < 0]
    encode_guess_correct_times2 = encode_guess_correct_times2.values[encode_guess_correct_times2.values >= 0]
    encode_guess_correct_times1 = np.array([encode_guess_correct_times1, np.ones(len(encode_guess_correct_times1)) * encoding_dur, np.ones(len(encode_guess_correct_times1))]).T
    encode_guess_correct_times2 = np.array([encode_guess_correct_times2, np.ones(len(encode_guess_correct_times2)) * encoding_dur, np.ones(len(encode_guess_correct_times2))]).T
    np.savetxt('onset_files/' + PID + '_encode_guess_correct_onset_run1.txt', encode_guess_correct_times1, fmt=form, delimiter=delim)
    np.savetxt('onset_files/' + PID + '_encode_guess_correct_onset_run2.txt', encode_guess_correct_times2, fmt=form, delimiter=delim)

    encode_guess_incorrect_times1 = frame.loc[((frame["trial_type"]=="Guess") & (frame["corr_recalled"]!=1)),"encode.started"] - triggers[0]
    encode_guess_incorrect_times2 = frame.loc[((frame["trial_type"]=="Guess") & (frame["corr_recalled"]!=1)),"encode.started"] - triggers[1]
    encode_guess_incorrect_times1 = encode_guess_incorrect_times1.values[encode_guess_incorrect_times2.values < 0]
    encode_guess_incorrect_times2 = encode_guess_incorrect_times2.values[encode_guess_incorrect_times2.values >= 0]
    encode_guess_incorrect_times1 = np.array([encode_guess_incorrect_times1, np.ones(len(encode_guess_incorrect_times1)) * encoding_dur, np.ones(len(encode_guess_incorrect_times1))]).T
    encode_guess_incorrect_times2 = np.array([encode_guess_incorrect_times2, np.ones(len(encode_guess_incorrect_times2)) * encoding_dur, np.ones(len(encode_guess_incorrect_times2))]).T
    np.savetxt('onset_files/' + PID + '_encode_guess_incorrect_onset_run1.txt', encode_guess_incorrect_times1, fmt=form, delimiter=delim)
    np.savetxt('onset_files/' + PID + '_encode_guess_incorrect_onset_run2.txt', encode_guess_incorrect_times2, fmt=form, delimiter=delim)

    # Read
    encode_read_correct_times1 = frame.loc[((frame["trial_type"]=="Read") & (frame["corr_recalled"]==1)),"encode.started"] - triggers[0]
    encode_read_correct_times2 = frame.loc[((frame["trial_type"]=="Read") & (frame["corr_recalled"]==1)),"encode.started"] - triggers[1]
    encode_read_correct_times1 = encode_read_correct_times1.values[encode_read_correct_times2.values < 0]
    encode_read_correct_times2 = encode_read_correct_times2.values[encode_read_correct_times2.values >= 0]
    encode_read_correct_times1 = np.array([encode_read_correct_times1, np.ones(len(encode_read_correct_times1)) * encoding_dur, np.ones(len(encode_read_correct_times1))]).T
    encode_read_correct_times2 = np.array([encode_read_correct_times2, np.ones(len(encode_read_correct_times2)) * encoding_dur, np.ones(len(encode_read_correct_times2))]).T
    np.savetxt('onset_files/' + PID + '_encode_read_correct_onset_run1.txt', encode_read_correct_times1, fmt=form, delimiter=delim)
    np.savetxt('onset_files/' + PID + '_encode_read_correct_onset_run2.txt', encode_read_correct_times2, fmt=form, delimiter=delim)

    encode_read_incorrect_times1 = frame.loc[((frame["trial_type"]=="Read") & (frame["corr_recalled"]!=1)),"encode.started"] - triggers[0]
    encode_read_incorrect_times2 = frame.loc[((frame["trial_type"]=="Read") & (frame["corr_recalled"]!=1)),"encode.started"] - triggers[1]
    encode_read_incorrect_times1 = encode_read_incorrect_times1.values[encode_read_incorrect_times2.values < 0]
    encode_read_incorrect_times2 = encode_read_incorrect_times2.values[encode_read_incorrect_times2.values >= 0]
    encode_read_incorrect_times1 = np.array([encode_read_incorrect_times1, np.ones(len(encode_read_incorrect_times1)) * encoding_dur, np.ones(len(encode_read_incorrect_times1))]).T
    encode_read_incorrect_times2 = np.array([encode_read_incorrect_times2, np.ones(len(encode_read_incorrect_times2)) * encoding_dur, np.ones(len(encode_read_incorrect_times2))]).T
    np.savetxt('onset_files/' + PID + '_encode_read_incorrect_onset_run1.txt', encode_read_incorrect_times1, fmt=form, delimiter=delim)
    np.savetxt('onset_files/' + PID + '_encode_read_incorrect_onset_run2.txt', encode_read_incorrect_times2, fmt=form, delimiter=delim)

1167
[128.0431454 537.153359 ]
2172
[104.4745847 506.7208945]
2405
[167.8826234 575.5783397]
3084
[142.7226727 547.9848248]
4110
[184.7474279 591.5245253]
5613
[125.7969   532.940891]
5968
[124.7301479 533.4241033]
6039
[141.2619062 544.7869617]
7382
[104.256487  506.1829734]
8424
[129.4602033 532.2710905]
8452
[146.8896881 565.4508436]
8752
[301.472086 702.332784]
9210
[122.0917897 524.4521897]


In [13]:
delim = ' '
form = '%.5f'

recall_dur = 3
recall_resp_dur = 1.5
recall_sel_dur = 3

for frame in recall_frames:
    PID = frame["PID"].values[0]
    triggers = frame["wait_for_trigger.stopped"][frame["wait_for_trigger.stopped"].notna()].values
    print(PID)
    print(triggers)

    if PID != "2020":
    # recall attempt - correct
    ## Guess
        recall_correct_guess1 = frame.loc[(frame["trial_type"]=="Guess") & (frame["correct_choice"]==1),"cued_recall.started"] - triggers[0]
        recall_correct_guess2 = frame.loc[(frame["trial_type"]=="Guess") & (frame["correct_choice"]==1),"cued_recall.started"] - triggers[1]
        recall_correct_guess1 = recall_correct_guess1.values[recall_correct_guess2.values < 0]
        recall_correct_guess2 = recall_correct_guess2.values[recall_correct_guess2.values >= 0]
        recall_correct_guess1 = np.array([recall_correct_guess1, np.ones(len(recall_correct_guess1)) * recall_dur, np.ones(len(recall_correct_guess1))]).T
        recall_correct_guess2 = np.array([recall_correct_guess2, np.ones(len(recall_correct_guess2)) * recall_dur, np.ones(len(recall_correct_guess2))]).T
        np.savetxt('onset_files/' + PID + '_correct_recall_guess_onset_run1.txt', recall_correct_guess1, fmt=form, delimiter=delim)
        np.savetxt('onset_files/' + PID + '_correct_recall_guess_onset_run2.txt', recall_correct_guess2, fmt=form, delimiter=delim)

        ## Read
        recall_correct_read1 = frame.loc[(frame["trial_type"]=="Read") & (frame["correct_choice"]==1),"cued_recall.started"] - triggers[0]
        recall_correct_read2 = frame.loc[(frame["trial_type"]=="Read") & (frame["correct_choice"]==1),"cued_recall.started"] - triggers[1]
        recall_correct_read1 = recall_correct_read1.values[recall_correct_read2.values < 0]
        recall_correct_read2 = recall_correct_read2.values[recall_correct_read2.values >= 0]
        recall_correct_read1 = np.array([recall_correct_read1, np.ones(len(recall_correct_read1)) * recall_dur, np.ones(len(recall_correct_read1))]).T
        recall_correct_read2 = np.array([recall_correct_read2, np.ones(len(recall_correct_read2)) * recall_dur, np.ones(len(recall_correct_read2))]).T
        np.savetxt('onset_files/' + PID + '_correct_recall_read_onset_run1.txt', recall_correct_read1, fmt=form, delimiter=delim)
        np.savetxt('onset_files/' + PID + '_correct_recall_read_onset_run2.txt', recall_correct_read2, fmt=form, delimiter=delim)
        
    # recall attempt - incorrect

        recall_incorrect_guess1 = frame.loc[(frame["trial_type"]=="Guess") & (frame["correct_choice"]==0),"cued_recall.started"] - triggers[0]
        recall_incorrect_guess2 = frame.loc[(frame["trial_type"]=="Guess") & (frame["correct_choice"]==0),"cued_recall.started"] - triggers[1]
        recall_incorrect_guess1 = recall_incorrect_guess1.values[recall_incorrect_guess2.values < 0]
        recall_incorrect_guess2 = recall_incorrect_guess2.values[recall_incorrect_guess2.values >= 0]
        recall_incorrect_guess1 = np.array([recall_incorrect_guess1, np.ones(len(recall_incorrect_guess1)) * recall_dur, np.ones(len(recall_incorrect_guess1))]).T
        recall_incorrect_guess2 = np.array([recall_incorrect_guess2, np.ones(len(recall_incorrect_guess2)) * recall_dur, np.ones(len(recall_incorrect_guess2))]).T
        np.savetxt('onset_files/' + PID + '_incorrect_recall_guess_onset_run1.txt', recall_incorrect_guess1, fmt=form, delimiter=delim)
        np.savetxt('onset_files/' + PID + '_incorrect_recall_guess_onset_run2.txt', recall_incorrect_guess2, fmt=form, delimiter=delim)

        recall_incorrect_read1 = frame.loc[(frame["trial_type"]=="Read") & (frame["correct_choice"]==0),"cued_recall.started"] - triggers[0]
        recall_incorrect_read2 = frame.loc[(frame["trial_type"]=="Read") & (frame["correct_choice"]==0),"cued_recall.started"] - triggers[1]
        recall_incorrect_read1 = recall_incorrect_read1.values[recall_incorrect_read2.values < 0]
        recall_incorrect_read2 = recall_incorrect_read2.values[recall_incorrect_read2.values >= 0]
        recall_incorrect_read1 = np.array([recall_incorrect_read1, np.ones(len(recall_incorrect_read1)) * recall_dur, np.ones(len(recall_incorrect_read1))]).T
        recall_incorrect_read2 = np.array([recall_incorrect_read2, np.ones(len(recall_incorrect_read2)) * recall_dur, np.ones(len(recall_incorrect_read2))]).T
        np.savetxt('onset_files/' + PID + '_incorrect_recall_read_onset_run1.txt', recall_incorrect_read1, fmt=form, delimiter=delim)
        np.savetxt('onset_files/' + PID + '_incorrect_recall_read_onset_run2.txt', recall_incorrect_read2, fmt=form, delimiter=delim)

    # recall response
        recall_response_guess_corr1 = frame.loc[(frame["trial_type"]=="Guess") & (frame["correct_choice"]==1), "recall_response.started"] - triggers[0]
        recall_response_guess_corr2 = frame.loc[(frame["trial_type"]=="Guess") & (frame["correct_choice"]==1), "recall_response.started"] - triggers[1]
        recall_response_guess_corr1 = recall_response_guess_corr1.values[recall_response_guess_corr2.values < 0]
        recall_response_guess_corr2 = recall_response_guess_corr2.values[recall_response_guess_corr2.values >= 0]
        recall_response_guess_corr1 = np.array([recall_response_guess_corr1, np.ones(len(recall_response_guess_corr1)) * recall_resp_dur, np.ones(len(recall_response_guess_corr1))]).T
        recall_response_guess_corr2 = np.array([recall_response_guess_corr2, np.ones(len(recall_response_guess_corr2)) * recall_resp_dur, np.ones(len(recall_response_guess_corr2))]).T
        np.savetxt('onset_files/' + PID + '_recall_response_guess_corr_onset_run1.txt', recall_response_guess_corr1, fmt=form, delimiter=delim)
        np.savetxt('onset_files/' + PID + '_recall_response_guess_corr_onset_run2.txt', recall_response_guess_corr2, fmt=form, delimiter=delim)

        recall_response_guess_incorr1 = frame.loc[(frame["trial_type"]=="Guess") & (frame["correct_choice"]==0), "recall_response.started"] - triggers[0]
        recall_response_guess_incorr2 = frame.loc[(frame["trial_type"]=="Guess") & (frame["correct_choice"]==0), "recall_response.started"] - triggers[1]
        recall_response_guess_incorr1 = recall_response_guess_incorr1.values[recall_response_guess_incorr2.values < 0]
        recall_response_guess_incorr2 = recall_response_guess_incorr2.values[recall_response_guess_incorr2.values >= 0]
        recall_response_guess_incorr1 = np.array([recall_response_guess_incorr1, np.ones(len(recall_response_guess_incorr1)) * recall_resp_dur, np.ones(len(recall_response_guess_incorr1))]).T
        recall_response_guess_incorr2 = np.array([recall_response_guess_incorr2, np.ones(len(recall_response_guess_incorr2)) * recall_resp_dur, np.ones(len(recall_response_guess_incorr2))]).T
        np.savetxt('onset_files/' + PID + '_recall_response_guess_incorr_onset_run1.txt', recall_response_guess_incorr1, fmt=form, delimiter=delim)
        np.savetxt('onset_files/' + PID + '_recall_response_guess_incorr_onset_run2.txt', recall_response_guess_incorr2, fmt=form, delimiter=delim)

        recall_response_read_corr1 = frame.loc[(frame["trial_type"]=="Read") & (frame["correct_choice"]==1), "recall_response.started"] - triggers[0]
        recall_response_read_corr2 = frame.loc[(frame["trial_type"]=="Read") & (frame["correct_choice"]==1), "recall_response.started"] - triggers[1]
        recall_response_read_corr1 = recall_response_read_corr1.values[recall_response_read_corr2.values < 0]
        recall_response_read_corr2 = recall_response_read_corr2.values[recall_response_read_corr2.values >= 0]
        recall_response_read_corr1 = np.array([recall_response_read_corr1, np.ones(len(recall_response_read_corr1)) * recall_resp_dur, np.ones(len(recall_response_read_corr1))]).T
        recall_response_read_corr2 = np.array([recall_response_read_corr2, np.ones(len(recall_response_read_corr2)) * recall_resp_dur, np.ones(len(recall_response_read_corr2))]).T
        np.savetxt('onset_files/' + PID + '_recall_response_read_corr_onset_run1.txt', recall_response_read_corr1, fmt=form, delimiter=delim)
        np.savetxt('onset_files/' + PID + '_recall_response_read_corr_onset_run2.txt', recall_response_read_corr2, fmt=form, delimiter=delim)

        recall_response_read_incorr1 = frame.loc[(frame["trial_type"]=="Read") & (frame["correct_choice"]==0), "recall_response.started"] - triggers[0]
        recall_response_read_incorr2 = frame.loc[(frame["trial_type"]=="Read") & (frame["correct_choice"]==0), "recall_response.started"] - triggers[1]
        recall_response_read_incorr1 = recall_response_read_incorr1.values[recall_response_read_incorr2.values < 0]
        recall_response_read_incorr2 = recall_response_read_incorr2.values[recall_response_read_incorr2.values >= 0]
        recall_response_read_incorr1 = np.array([recall_response_read_incorr1, np.ones(len(recall_response_read_incorr1)) * recall_resp_dur, np.ones(len(recall_response_read_incorr1))]).T
        recall_response_read_incorr2 = np.array([recall_response_read_incorr2, np.ones(len(recall_response_read_incorr2)) * recall_resp_dur, np.ones(len(recall_response_read_incorr2))]).T
        np.savetxt('onset_files/' + PID + '_recall_response_read_incorr_onset_run1.txt', recall_response_read_incorr1, fmt=form, delimiter=delim)
        np.savetxt('onset_files/' + PID + '_recall_response_read_incorr_onset_run2.txt', recall_response_read_incorr2, fmt=form, delimiter=delim)

        # recall selection
        recall_selection_corr1 = frame.loc[(frame["correct_choice"]==1), "recall_selection.started"] - triggers[0]
        recall_selection_corr2 = frame.loc[(frame["correct_choice"]==1), "recall_selection.started"] - triggers[1]
        recall_selection_corr1 = recall_selection_corr1.values[recall_selection_corr2.values < 0]
        recall_selection_corr2 = recall_selection_corr2.values[recall_selection_corr2.values >= 0]
        recall_selection_corr1 = np.array([recall_selection_corr1, np.ones(len(recall_selection_corr1)) * recall_sel_dur, np.ones(len(recall_selection_corr1))]).T
        recall_selection_corr2 = np.array([recall_selection_corr2, np.ones(len(recall_selection_corr2)) * recall_sel_dur, np.ones(len(recall_selection_corr2))]).T
        np.savetxt('onset_files/' + PID + '_recall_selection_corr_onset_run1.txt', recall_selection_corr1, fmt=form, delimiter=delim)
        np.savetxt('onset_files/' + PID + '_recall_selection_corr_onset_run2.txt', recall_selection_corr2, fmt=form, delimiter=delim)

        recall_selection_incorr1 = frame.loc[(frame["correct_choice"]==0), "recall_selection.started"] - triggers[0]
        recall_selection_incorr2 = frame.loc[(frame["correct_choice"]==0), "recall_selection.started"] - triggers[1]
        recall_selection_incorr1 = recall_selection_incorr1.values[recall_selection_incorr2.values < 0]
        recall_selection_incorr2 = recall_selection_incorr2.values[recall_selection_incorr2.values >= 0]
        recall_selection_incorr1 = np.array([recall_selection_incorr1, np.ones(len(recall_selection_incorr1)) * recall_sel_dur, np.ones(len(recall_selection_incorr1))]).T
        recall_selection_incorr2 = np.array([recall_selection_incorr2, np.ones(len(recall_selection_incorr2)) * recall_sel_dur, np.ones(len(recall_selection_incorr2))]).T
        np.savetxt('onset_files/' + PID + '_recall_selection_incorr_onset_run1.txt', recall_selection_incorr1, fmt=form, delimiter=delim)
        np.savetxt('onset_files/' + PID + '_recall_selection_incorr_onset_run2.txt', recall_selection_incorr2, fmt=form, delimiter=delim)
    
    else:
        continue

1167
[1098.2882933 1528.3643913]
2172
[1050.1070906 1515.3476555]
2405
[1218.9254931 1649.232977 ]
3084
[1242.5285599 1689.7033005]
4110
[1238.8549187 1676.34916  ]
5613
[1181.9194384 1601.445035 ]
5968
[1110.7736432 1531.7685277]
6039
[1231.5653294 1617.5932165]
7382
[1056.0997427 1489.1412296]
8424
[1112.954844  1536.9309094]
8452
[1217.4804033 1624.2398569]
8752
[1284.9492884 1719.8892558]
9210
[1091.4535152 1569.9599906]
